# **Libraries**

In [ ]:
!pip install ultralytics

# **Prediction**

In [ ]:
import os
import glob
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

try:
    # 1. Load your YOLO model
    print("Loading model...")
    model = YOLO("best.pt")
    print("Model loaded successfully!")

    # 2. Start the interactive loop
    while True:
        image_path = input("Enter the path to your image, or type 'q' to quit: ")

        if image_path.lower() == 'q':
            print("Exiting...")
            break

        if not os.path.exists(image_path):
            print(f"\n❌ Error: File not found at '{image_path}'.")
            print("Hint: Right-click the file in the sidebar and select 'Copy path'.\n")
            continue

        # 3. Run prediction on the image
        print(f"\nAnalyzing '{os.path.basename(image_path)}'...")
        results = model.predict(source=image_path, save=True, conf=0.2)

        # 4. Print the detected emotions as text
        print("\n--- DETECTED EMOTIONS ---")
        if results and results[0].boxes:
            names = model.names
            for box in results[0].boxes:
                class_id = int(box.cls[0])
                class_name = names[class_id]
                confidence = box.conf[0]
                print(f"Emotion: {class_name}, Confidence: {confidence:.2f}")
        else:
            print("No emotions detected.")
        print("-------------------------\n")

        # 5. Display the result image using Matplotlib (the robust way)
        latest_predict_folder = max(glob.glob(os.path.join('runs/detect', 'predict*')), key=os.path.getmtime)
        result_image_path = os.path.join(latest_predict_folder, os.path.basename(image_path))

        if os.path.exists(result_image_path):
            # Read the image using OpenCV
            img = cv2.imread(result_image_path)
            # Convert it from BGR (OpenCV's default) to RGB (Matplotlib's default)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Display the image
            plt.figure(figsize=(8, 6))
            plt.imshow(img_rgb)
            plt.axis('off') # Hide the axes
            plt.show()
        else:
            print("Could not find the result image to display.")

except Exception as err:
    print(f"An error occurred: {err}")